In [ ]:
# Translate AKI_AMED_POSIX to csv
# site: IUR, UNMC, UPITT, UTSW
library(rxnorm)
library(dplyr)
library(data.table)
library(reshape2)
library(stringi)
site <- 'UTSW'
home_directory <- paste0('/home/hchan2/AKI/data/',site,'/raw/')
amed <- readRDS(paste0(home_directory,"AKI_AMED_POSIX.rda"))
write.csv(amed, paste0(home_directory,"AKI_AMED.csv"))

In [ ]:
#library(devtools)
#devtools::install_github("fabkury/ndc_map")
install.packages("tcltk", repos='http://cran.us.r-project.org')

In [ ]:
print('done')

In [ ]:
install.packages("mgcv", repos='http://cran.us.r-project.org')

In [ ]:
tl <- list()

In [ ]:
tl['tt2'] <- data.frame('aa', 'bb')

In [ ]:

siteyrlist <- {}
siteyrlist$foo$bar1 <- 1
siteyrlist$foo$bar2 <- 2

In [ ]:
library(arrow)
library(dplyr)
library(mgcv)
library(tidyverse)
library(data.table)
library(metagam)
library(latex2exp)
library(eivtools)
library(plotly)
library(stats)
library(plotrix)
library(fishmethods)
library(metafor)

In [ ]:
plot_fit2<-function(df2, gamobj, coeff){
    
    df2<- df2 %>% mutate(site = substr(siteyr,1,nchar(siteyr)-4))
    #    df2<-df2 %>%
    #        mutate(lwr = df2[[coeff]] - 2 * df2[[paste0(coeff,'_var')]],
    #         upr = df2[[coeff]] + 2 * df2[[paste0(coeff,'_var')]])    

    fit1_FIT<-predict(gamobj, se.fit=T) %>%
      as_tibble() %>% 
      rename(fit_gam = fit) %>% 
      mutate(lwr_gam = fit_gam - 2 * se.fit,
             upr_gam = fit_gam + 2 * se.fit)

    ggplot()+
    geom_point(aes(df2[[coeff]], df2$auc, color=df2$site))+
    geom_line(aes(df2[[coeff]], fit1_FIT$fit_gam, color='red'))+
    geom_ribbon(aes(x=df2[[coeff]], ymin=fit1_FIT$lwr_gam, ymax=fit1_FIT$upr_gam), alpha = 0.2, fill = "red")
#    geom_errorbarh(aes(y=df2$auc, xmin=df2$lwr, xmax=df2$upr), alpha = 0.2)    
}

In [ ]:
path <- "metadata_mean.parquet"
df <- arrow::read_parquet(path)
path <- "metadata_raw.parquet"
dfraw <- arrow::read_parquet(path)

In [ ]:
df = df %>% dplyr::select(-'__index_level_0__') %>% mutate(siteyr=paste0(site,year)) %>% filter(Importances!=0)

#df = df %>% filter(!(site %in% list("MCRI"))) %>% filter(!(sy %in% list("UMHC2013", "UMHC2020")))
df = df %>% filter(!(site %in% list("MCRI", 'UPITT'))) %>% filter(!(siteyr %in% list("UMHC2020")))

siteyrlist = df %>% dplyr::select(siteyr) %>% unique
featuredecodelist = df %>% dplyr::select(featuredecode) %>% unique
dfraw = dfraw %>% rename(fval=Name) %>% rename(siteyr=siteyear) %>% dplyr::select(-'__index_level_0__')

In [ ]:
filterdf <- df %>% 
    select(site,year,featuredecode,fval) %>% 
    group_by(site,year,featuredecode) %>% 
    summarise(min=min(fval), max=max(fval)) %>%
    ungroup %>%
    group_by(featuredecode) %>%
    summarise(min_min=min(min), max_min=max(min), min_max=min(max), max_max=max(max), mean_min=mean(min), sd_min=sd(min), mean_max=mean(max), sd_max=sd(max)) %>%
    mutate(upr=mean_max+3*sd_max, lwr=mean_min-3*sd_min) %>% 
    mutate(upr = replace(upr, featuredecode == "AGE", 89)) %>% 
    mutate(lwr = replace(lwr, featuredecode == "AGE", 18))

df <- df %>% left_join(filterdf%>%select(featuredecode,upr,lwr), by='featuredecode') %>% filter(fval>=lwr) %>% filter(fval<=upr) %>% select(-upr, -lwr)
dfraw <- dfraw %>% left_join(filterdf%>%select(featuredecode,upr,lwr), by='featuredecode') %>% filter(fval>=lwr) %>% filter(fval<=upr) %>% select(-upr, -lwr)

In [ ]:
frange <- dfraw %>% select(fval, featuredecode) %>% group_by(featuredecode) %>% 
                        summarize(min=min(fval),q10 =quantile(fval,probs=0.001),
                                                q25 =quantile(fval,probs=0.25),
                                                q375=quantile(fval,probs=0.375),
                                                q50 =quantile(fval,probs=0.50),
                                                q625=quantile(fval,probs=0.625),
                                                q75 =quantile(fval,probs=0.75),
                                                q90 =quantile(fval,probs=0.999),
                                                max=max(fval))
frange

In [ ]:
# fit curve at theior own domain
# ptm <- proc.time()
# gamdict <- list()
# for (sy in siteyrlist$siteyr){
#     dfrawsy <- dfraw %>% filter(siteyr==sy)
#     syflist <- dfrawsy %>% select(featuredecode) %>% unique
#     gamsydict <- list()
#     for (syf in syflist$featuredecode){
# #        print(paste0(sy,',',syf))
# #        flush.console()
#         dftt <- dfrawsy %>% filter(featuredecode==syf)
#         if (!all(dftt['val']==0)){
#             x<-list(gam(val ~ s(fval, k=40, bs='cs'), data=dftt, method='REML'))
#             names(x) <- syf
#             gamsydict <- append(gamsydict,x)
#         }
#     }
#     gamsydict2<-list(gamsydict)
#     names(gamsydict2) <- sy
#     gamdict <- append(gamdict,gamsydict2)
# }
# proc.time() - ptm

#saveRDS(gamdict, file = "gamfitobj.rds")
#gamdict<-readRDS(file = "gamfitobj.rds")

#Get the value at equal distance
# fitlist = list()
# for (sy in siteyrlist$siteyr){
#     for (syf in frange$featuredecode){
#         fr<-frange %>% filter(featuredecode==syf)
#         newd<- data.frame(fval=seq(from=fr$min, to=fr$max, length.out=10))
#         x<-gamdict[[sy]][[syf]]
#         if (!is.null(x)){
#             fit1_FIT <- list(data.frame(predict(x, newd, se.fit=T) %>% as_tibble() %>% mutate(siteyr=sy,featuredecode=syf),newd))
#             fitlist <- append(fitlist,fit1_FIT)
#         }
#     }
# }
# fitlist<-rbindlist(fitlist) %>% mutate(fval=round(fval,2)) %>% mutate(fval=paste0(featuredecode,'.',fval))
# fitlist<-fitlist %>% mutate(featuredecode=str_replace(featuredecode,'-','.')) %>% mutate(featuredecode=paste0('f',featuredecode))
# fitlist<-fitlist %>% mutate(fval=str_replace(fval,'-','.')) %>% mutate(fval=paste0('f',fval))

In [ ]:
#Alternatively fit and get coefficient

#Make Average to be zero?
#dfraw <- dfraw %>% group_by(siteyr, Feature) %>% mutate(val=val-mean(val)) %>% ungroup

ptm <- proc.time()
gamdict <- list()
fitlist <- list()
#sy <- 'KUMC2013'
#syf <- 'SYSTOLIC'
for (sy in siteyrlist$siteyr){
#    site <- substr(sy, 0, nchar(sy)-4)    
#    if ((site %in% list("MCRI")) | sy %in% list("UMHC2013", "UMHC2020")){
#    if ((site %in% list("MCRI")) | sy %in% list("UMHC2013", "UMHC2020")){    
#        next
#    }    
    dfrawsy <- dfraw %>% filter(siteyr==sy)
    syflist <- dfrawsy %>% select(featuredecode) %>% unique
    gamsydict <- list()
    for (syf in syflist$featuredecode){
        print(paste0(sy,'-',syf))
        flush.console()
        dftt <- dfrawsy %>% filter(featuredecode==syf)
        fr<-frange %>% filter(featuredecode==syf)
        if (!all(dftt['val']==0)){
            x<-list(gam(val ~ s(fval, k=6, bs='cr'), data=dftt, method='REML', knots = list(fval = seq(from=fr$min, to=fr$max, length = 6))))
            names(x) <- syf
            gamsydict <- append(gamsydict,x)
            
            rangelist <- list()
            for (i in 1:length(x[[syf]]$smooth[[1]]$xp)){
                rangelist <- c(rangelist, paste0('r', x[[syf]]$smooth[[1]]$xp[i], 'r', x[[syf]]$smooth[[1]]$xp[i+1], 'r'))
            }
            rangelist <- rangelist[-length(rangelist)]
            rangelist <- c('intercept', rangelist)
            coeffdata <- cbind(data.frame(x[[syf]]$coefficient), data.frame(diag(x[[syf]]$Vc)), data.frame(transpose(rangelist)))
            colnames(coeffdata) <- c('fit', 'se.fit', 'fval')
            coeffdata <- coeffdata %>% mutate(siteyr=sy,featuredecode=syf)
            coeffdata <- coeffdata[-1,]
            coeffdata <- coeffdata %>% mutate(fval=paste0(featuredecode,'.',fval))
            row.names(coeffdata) <- NULL
            fitlist <- append(fitlist,list(coeffdata))            
        }
    }
#    gamsydict2<-list(gamsydict)
#    names(gamsydict2) <- sy
#    gamdict <- append(gamdict,gamsydict2)
}
proc.time() - ptm
#gamdict<-rbindlist(gamdict)
fitlist<-rbindlist(fitlist) %>% mutate(featuredecode=str_replace(featuredecode,'-','d')) %>% mutate(featuredecode=paste0('f',featuredecode)) %>% mutate(fval=str_replace_all(fval,'-','d')) %>% mutate(fval=paste0('f',fval))